In [31]:
from cqlib import TianYanPlatform  # 导出量子计算机SDK的支持包
from cqlib.utils import QasmToQcis
from datetime import datetime
from qiskit import QuantumCircuit
import random
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from cqlib.utils import LaboratoryUtils
import re
import numpy as np
import math
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit import QuantumCircuit, execute
import pickle
login_key = "+I771T2LACBkZE8CYnvTiWjtX48eTh0FxNja3OUEaxI="



# 创建实例，设置用户SDK密钥，选择量子计算机，此处为示例，选择的是应答机而非真实的量子计算机
# 用户使用时，请务必用自己的密钥更新上述内容，否则上述密钥会被定时更新而示例程序不可以。
platform = TianYanPlatform(login_key=login_key)


# 查询量子计算机列表
computer_list_data = platform.query_quantum_computer_list()
for computer_data in computer_list_data:
    print(computer_data)


# 设置使用仿真机，此为示例，实际使用时请选择正确的量子计算机。
platform.set_machine("tianyan176")

['3', 'free', 'running', 'zdxlz_simulator']
['4', 'free', 'running', 'stabilizer']
['1764555284795101186', 'free', 'running', 'xuanyuanone']


In [32]:

lab_id = platform.create_lab(name=f'lab.{datetime.now().strftime("%Y%m%d%H%M%S")}', remark='test_collection')
lab_id

1817887862194884609

In [4]:
# 手动提取赋值
# topography = [[0, 6], [0, 7], [1, 7], [1, 8], [2, 8], [2, 9], [3, 9], [3, 10], [4, 10], [4, 11], [6, 12], [7, 12],
#               [7, 13], [8, 13], [8, 14], [9, 14], [9, 15], [10, 15], [10, 16], [11, 16], [11, 17], [12, 18], [12, 19], 
#               [13, 19], [13, 20], [14, 20], [14, 21], [15, 21], [15, 22], [16, 22], [16, 23], [17, 23], [18, 24], [19, 24],
#               [19, 25], [20, 25], [20, 26], [21, 26], [21, 27], [22, 27], [22, 28], [23, 28], [23, 29], [24, 30], [24, 31], 
#               [25, 31], [25, 32], [26, 32], [26, 33], [27, 33], [27, 34], [28, 34], [30, 36], [31, 36], [31, 37], [32, 37], 
#               [32, 38], [33, 38], [33, 39], [34, 39], [34, 40], [36, 42], [36, 43], [37, 43], [37, 44], [38, 44], [38, 45],
#               [39, 45], [39, 46], [40, 46], [42, 48], [43, 48], [43, 49], [44, 49], [44, 50], [45, 50], [45, 51], [46, 51], 
#               [46, 52], [48, 54], [48, 55], [49, 55], [49, 56], [50, 56], [50, 57], [51, 57], [51, 58], [52, 58], [52, 59],
#               [53, 59], [54, 60], [55, 60], [55, 61], [56, 61], [56, 62], [57, 62], [57, 63], [58, 63], [58, 64], [59, 64],
#               [59, 65]]

topography_rename = {}
k = 0
for i in range(5):
    for j in range(6):
        if j<=4:
            topography_rename[(12*i+j,12*i+j+6)] = "G"+str(k)
            topography_rename[(12*i+j,12*i+j+7)] = "G"+str(k+1)
            k+=2
        else:
            topography_rename[(12*i+j,12*i+j+6)] = "G"+str(k)
            k+=1
    for j in range(6):
        if j<=4:
            topography_rename[(12*i+j+6,12*i+j+12)] = "G"+str(k)
            topography_rename[(12*i+j+7,12*i+j+12)] = "G"+str(k+1)
            k+=2
        else:
            topography_rename[(12*i+j+6,12*i+j+12)] = "G"+str(k)
            k+=1
            
            
            


lu = LaboratoryUtils()
config_save = platform.download_config()





props = {}
# for key, value in config_save['twoQubitGate']['czGate'].items():
#     print(key, value)

print(config_save['twoQubitGate']['czGate']['gate error']['update_time']) #校准时间
CZ_error = config_save['twoQubitGate']['czGate']['gate error']['param_list'] #CZ错误率
# print(CZ_error)
CZ_name = config_save['twoQubitGate']['czGate']['gate error']['qubit_used']  #相关的CZ门名称，与topography_rename相对应
# print(CZ_name)
CZ_error_list = {}  # cz gate_error dict
for key, value in topography_rename.items():
    if value in CZ_name:
        CZ_error_list[(key[0],key[1])] = CZ_error[CZ_name.index(value)]
        CZ_error_list[(key[1],key[0])] = CZ_error[CZ_name.index(value)]
    else:
        CZ_error_list[(key[0],key[1])] = np.nan
        CZ_error_list[(key[1],key[0])] = np.nan

singleQubit_error_dict = {}
singleQubit_readoutError_dict = {}
singleQubit_error = config_save['qubit']['singleQubit']['gate error']['param_list']
singleQubit_list = config_save['qubit']['singleQubit']['gate error']['qubit_used']
singleQubit_readoutErrorlist = config_save['readout']['readoutArray']['Readout Error']['param_list']
# 正则表达式，匹配所有形式的数字
pattern = r'\d+'
for i in range(len(singleQubit_list)):
    number = int(re.findall(pattern, singleQubit_list[i])[0])
    singleQubit_error_dict[number] = singleQubit_error[i]
    singleQubit_readoutError_dict[number] = singleQubit_readoutErrorlist[i]
# singleQubit_error_dict
props['singleQubit_error'] = singleQubit_error_dict 
# singleQubit_readoutError_dict
props['singleQubit_readoutError'] = singleQubit_readoutError_dict 
props['CZ_error'] = CZ_error_list
props
overview_cz_error = config_save['overview']['cz_error']  #两比特门错误率
overview_single_error = config_save['overview']['1q_gate_error'] #单比特门错误率
overview_readout_error = config_save['overview']['readout_error']#读取错误率
# print(overview_cz_error,overview_single_error ,overview_readout_error )


# 获取后端拓扑

coupling_map = []
for map in CZ_error_list:
    if not math.isnan(CZ_error_list[map]):
        coupling_map.append([map[0],map[1]])
print(coupling_map)


# 获取可用的比特号
def get_unique_numbers(coupling_map):
    unique_numbers = set()
    for pair in coupling_map:
        unique_numbers.update(pair)
    return list(unique_numbers)
useful_qubits = get_unique_numbers(coupling_map)
print(useful_qubits)

2024-07-29 18:23:34
[[0, 6], [6, 0], [0, 7], [7, 0], [1, 7], [7, 1], [1, 8], [8, 1], [2, 8], [8, 2], [3, 9], [9, 3], [3, 10], [10, 3], [4, 10], [10, 4], [4, 11], [11, 4], [6, 12], [12, 6], [7, 12], [12, 7], [7, 13], [13, 7], [8, 13], [13, 8], [8, 14], [14, 8], [9, 14], [14, 9], [9, 15], [15, 9], [10, 15], [15, 10], [12, 18], [18, 12], [12, 19], [19, 12], [13, 19], [19, 13], [14, 20], [20, 14], [14, 21], [21, 14], [18, 24], [24, 18], [19, 24], [24, 19], [19, 25], [25, 19], [20, 25], [25, 20], [20, 26], [26, 20], [21, 26], [26, 21], [21, 27], [27, 21], [22, 28], [28, 22], [23, 29], [29, 23], [24, 30], [30, 24], [24, 31], [31, 24], [25, 31], [31, 25], [25, 32], [32, 25], [26, 32], [32, 26], [26, 33], [33, 26], [27, 33], [33, 27], [27, 34], [34, 27], [28, 34], [34, 28], [30, 36], [36, 30], [31, 36], [36, 31], [31, 37], [37, 31], [32, 37], [37, 32], [32, 38], [38, 32], [33, 38], [38, 33], [33, 39], [39, 33], [34, 39], [39, 34], [34, 40], [40, 34], [36, 42], [42, 36], [36, 43], [43, 36], [37

In [4]:
print(CZ_error_list)
print(singleQubit_error_dict)
print(singleQubit_readoutError_dict)

{(0, 6): 1.38, (6, 0): 1.38, (0, 7): 2.02, (7, 0): 2.02, (1, 7): 1.22, (7, 1): 1.22, (1, 8): 1.42, (8, 1): 1.42, (2, 8): 1.45, (8, 2): 1.45, (2, 9): nan, (9, 2): nan, (3, 9): 1.62, (9, 3): 1.62, (3, 10): 1.4, (10, 3): 1.4, (4, 10): 2.49, (10, 4): 2.49, (4, 11): 0.99, (11, 4): 0.99, (5, 11): nan, (11, 5): nan, (6, 12): 2.03, (12, 6): 2.03, (7, 12): 5.36, (12, 7): 5.36, (7, 13): 1.7, (13, 7): 1.7, (8, 13): 3.66, (13, 8): 3.66, (8, 14): 1.42, (14, 8): 1.42, (9, 14): 0.89, (14, 9): 0.89, (9, 15): 2.19, (15, 9): 2.19, (10, 15): 1.07, (15, 10): 1.07, (10, 16): nan, (16, 10): nan, (11, 16): nan, (16, 11): nan, (11, 17): nan, (17, 11): nan, (12, 18): 1.74, (18, 12): 1.74, (12, 19): 3.1, (19, 12): 3.1, (13, 19): 2.13, (19, 13): 2.13, (13, 20): nan, (20, 13): nan, (14, 20): 1.35, (20, 14): 1.35, (14, 21): 2.13, (21, 14): 2.13, (15, 21): nan, (21, 15): nan, (15, 22): nan, (22, 15): nan, (16, 22): nan, (22, 16): nan, (16, 23): nan, (23, 16): nan, (17, 23): nan, (23, 17): nan, (18, 24): 2.12, (24, 

In [5]:
converted_dict = {str(key): value for key, value in CZ_error_list.items()}

In [6]:
converted_dict

{'(0, 6)': 1.51,
 '(6, 0)': 1.51,
 '(0, 7)': 2.65,
 '(7, 0)': 2.65,
 '(1, 7)': 1.29,
 '(7, 1)': 1.29,
 '(1, 8)': 1.27,
 '(8, 1)': 1.27,
 '(2, 8)': 1.17,
 '(8, 2)': 1.17,
 '(2, 9)': nan,
 '(9, 2)': nan,
 '(3, 9)': 1.7,
 '(9, 3)': 1.7,
 '(3, 10)': 1.54,
 '(10, 3)': 1.54,
 '(4, 10)': 2.83,
 '(10, 4)': 2.83,
 '(4, 11)': 1.0,
 '(11, 4)': 1.0,
 '(5, 11)': nan,
 '(11, 5)': nan,
 '(6, 12)': 1.76,
 '(12, 6)': 1.76,
 '(7, 12)': 2.65,
 '(12, 7)': 2.65,
 '(7, 13)': 1.78,
 '(13, 7)': 1.78,
 '(8, 13)': 2.4,
 '(13, 8)': 2.4,
 '(8, 14)': 1.6,
 '(14, 8)': 1.6,
 '(9, 14)': 0.88,
 '(14, 9)': 0.88,
 '(9, 15)': 2.25,
 '(15, 9)': 2.25,
 '(10, 15)': 0.88,
 '(15, 10)': 0.88,
 '(10, 16)': nan,
 '(16, 10)': nan,
 '(11, 16)': nan,
 '(16, 11)': nan,
 '(11, 17)': nan,
 '(17, 11)': nan,
 '(12, 18)': 1.8,
 '(18, 12)': 1.8,
 '(12, 19)': 2.27,
 '(19, 12)': 2.27,
 '(13, 19)': 1.93,
 '(19, 13)': 1.93,
 '(13, 20)': nan,
 '(20, 13)': nan,
 '(14, 20)': 1.34,
 '(20, 14)': 1.34,
 '(14, 21)': 1.58,
 '(21, 14)': 1.58,
 '(15, 2

In [8]:
import json
tianya_error = {'CZ_error':converted_dict,'singleQubit_error':singleQubit_error_dict,'singleQubit_readoutError':singleQubit_readoutError_dict}
print(tianya_error)
with open('./multi_backend_model_data/tianyan_error.json', 'w') as file:
        json.dump(tianya_error, file)

{'CZ_error': {'(0, 6)': 1.51, '(6, 0)': 1.51, '(0, 7)': 2.65, '(7, 0)': 2.65, '(1, 7)': 1.29, '(7, 1)': 1.29, '(1, 8)': 1.27, '(8, 1)': 1.27, '(2, 8)': 1.17, '(8, 2)': 1.17, '(2, 9)': nan, '(9, 2)': nan, '(3, 9)': 1.7, '(9, 3)': 1.7, '(3, 10)': 1.54, '(10, 3)': 1.54, '(4, 10)': 2.83, '(10, 4)': 2.83, '(4, 11)': 1.0, '(11, 4)': 1.0, '(5, 11)': nan, '(11, 5)': nan, '(6, 12)': 1.76, '(12, 6)': 1.76, '(7, 12)': 2.65, '(12, 7)': 2.65, '(7, 13)': 1.78, '(13, 7)': 1.78, '(8, 13)': 2.4, '(13, 8)': 2.4, '(8, 14)': 1.6, '(14, 8)': 1.6, '(9, 14)': 0.88, '(14, 9)': 0.88, '(9, 15)': 2.25, '(15, 9)': 2.25, '(10, 15)': 0.88, '(15, 10)': 0.88, '(10, 16)': nan, '(16, 10)': nan, '(11, 16)': nan, '(16, 11)': nan, '(11, 17)': nan, '(17, 11)': nan, '(12, 18)': 1.8, '(18, 12)': 1.8, '(12, 19)': 2.27, '(19, 12)': 2.27, '(13, 19)': 1.93, '(19, 13)': 1.93, '(13, 20)': nan, '(20, 13)': nan, '(14, 20)': 1.34, '(20, 14)': 1.34, '(14, 21)': 1.58, '(21, 14)': 1.58, '(15, 21)': nan, '(21, 15)': nan, '(15, 22)': nan,

In [9]:
with open('./multi_backend_model_data/tianyan_error.json', 'r') as file:
        data = json.load(file)
data

{'CZ_error': {'(0, 6)': 1.51,
  '(6, 0)': 1.51,
  '(0, 7)': 2.65,
  '(7, 0)': 2.65,
  '(1, 7)': 1.29,
  '(7, 1)': 1.29,
  '(1, 8)': 1.27,
  '(8, 1)': 1.27,
  '(2, 8)': 1.17,
  '(8, 2)': 1.17,
  '(2, 9)': nan,
  '(9, 2)': nan,
  '(3, 9)': 1.7,
  '(9, 3)': 1.7,
  '(3, 10)': 1.54,
  '(10, 3)': 1.54,
  '(4, 10)': 2.83,
  '(10, 4)': 2.83,
  '(4, 11)': 1.0,
  '(11, 4)': 1.0,
  '(5, 11)': nan,
  '(11, 5)': nan,
  '(6, 12)': 1.76,
  '(12, 6)': 1.76,
  '(7, 12)': 2.65,
  '(12, 7)': 2.65,
  '(7, 13)': 1.78,
  '(13, 7)': 1.78,
  '(8, 13)': 2.4,
  '(13, 8)': 2.4,
  '(8, 14)': 1.6,
  '(14, 8)': 1.6,
  '(9, 14)': 0.88,
  '(14, 9)': 0.88,
  '(9, 15)': 2.25,
  '(15, 9)': 2.25,
  '(10, 15)': 0.88,
  '(15, 10)': 0.88,
  '(10, 16)': nan,
  '(16, 10)': nan,
  '(11, 16)': nan,
  '(16, 11)': nan,
  '(11, 17)': nan,
  '(17, 11)': nan,
  '(12, 18)': 1.8,
  '(18, 12)': 1.8,
  '(12, 19)': 2.27,
  '(19, 12)': 2.27,
  '(13, 19)': 1.93,
  '(19, 13)': 1.93,
  '(13, 20)': nan,
  '(20, 13)': nan,
  '(14, 20)': 1.34,


In [33]:

def generate_random_circuit(coupling_map,useful_qubits, num_qubits,num_cbits, depth, gate_set):
    # 找出可能的量子比特对
    selected_qubits = set()
    valid_gates = []
    while len(selected_qubits) < num_cbits/2:
        gate = random.choice(coupling_map)
        selected_qubits.add(gate[0])
        selected_qubits.add(gate[1])
        valid_gates.append([gate[0],gate[1]])
    
    # 若量子比特数不足，再随机添加一个量子比特
    while len(selected_qubits) < num_cbits:
        new_qubit = random.choice(useful_qubits)
        selected_qubits.add(new_qubit)
    selected_qubits = list(selected_qubits)
    # 初始化量子线路和对应的经典寄存器
    circuit = QuantumCircuit(num_qubits,num_cbits)

    # print(circuit)
    # print(selected_qubits)
    # 添加单比特门和2比特门
    for _ in range(depth):
        for qubit in selected_qubits:
            gate = random.choice(gate_set)
            if gate == 'h':
                circuit.h(qubit)
            elif gate == 'x':
                circuit.x(qubit)
            elif gate == 'sx':
                circuit.sx(qubit)
            elif gate == 'rz':
                angle = random.uniform(-3.1415926, 3.1415926)  # 随机选择一个角度
                circuit.rz(angle, qubit)
            elif gate == 'cx':
                [control, target] = random.choice(valid_gates)
                circuit.cx(control, target)
            elif gate == 'id':
                continue

    # 测量所有的量子比特
    for cbit in range(num_cbits):
        circuit.measure(selected_qubits[cbit], cbit)

    return circuit

In [41]:

def generate_random_circuit_25(coupling_map,useful_qubits, num_qubits,num_cbits, depth, gate_set):
    # 找出可能的量子比特对
    selected_qubits = set()
    valid_gates = []
    while len(selected_qubits) < num_cbits/2:
        gate = random.choice(coupling_map)
        selected_qubits.add(gate[0])
        selected_qubits.add(gate[1])
        valid_gates.append([gate[0],gate[1]])
    
    # 若量子比特数不足，再随机添加一个量子比特
    while len(selected_qubits) < num_cbits:
        new_qubit = random.choice(useful_qubits)
        selected_qubits.add(new_qubit)
    selected_qubits = list(selected_qubits)
    # 初始化量子线路和对应的经典寄存器
    circuit = QuantumCircuit(num_qubits,num_cbits)
    # print(circuit)
    # print(selected_qubits)
    # 添加单比特门和2比特门
    count = 0  # 计数器
    for _ in range(depth):
        for qubit in selected_qubits:
            if count >= 10:  # 如果门的数量达到10个，跳出循环
                break
            gate = random.choice(gate_set)
            if gate == 'h':
                circuit.h(qubit)
                count += 1
            elif gate == 'x':
                circuit.x(qubit)
                count += 1
            elif gate == 'sx':
                circuit.sx(qubit)
                count += 1
            elif gate == 'rz':
                angle = random.uniform(-3.1415926, 3.1415926)  # 随机选择一个角度
                circuit.rz(angle, qubit)
                count += 1
            elif gate == 'cx':
                [control, target] = random.choice(valid_gates)
                circuit.cx(control, target)
                count += 1
            elif gate == 'id':
                continue
            

    # 测量所有的量子比特
    for cbit in range(num_cbits):
        circuit.measure(selected_qubits[cbit], cbit)

    return circuit

In [34]:
def run_tianyan_circuit(qcis_circuit,platform,lab_id):
    query_id_1 = platform.submit_job(
        circuit=qcis_circuit,
        exp_name=f'exp.{datetime.now().strftime("%Y%m%d%H%M%S")}',
        lab_id=lab_id,
        num_shots=100000,
    )
    query_id_2 = platform.submit_job(
            circuit=qcis_circuit,
            exp_name=f'exp.{datetime.now().strftime("%Y%m%d%H%M%S")}',
            lab_id=lab_id,
            num_shots=100000,
        )
    query_id_3 = platform.submit_job(
            circuit=qcis_circuit,
            exp_name=f'exp.{datetime.now().strftime("%Y%m%d%H%M%S")}',
            lab_id=lab_id,
            num_shots=100000,
        )
    exp_result_1 = platform.query_experiment(query_id=query_id_1, max_wait_time=10000, sleep_time=0)
    exp_result_2 = platform.query_experiment(query_id=query_id_2, max_wait_time=10000, sleep_time=0)
    exp_result_3 = platform.query_experiment(query_id=query_id_3, max_wait_time=10000, sleep_time=0)
    lu = LaboratoryUtils()
    #将结果的全部空间进行统计
    noisy_value_1=lu.readout_data_to_state_probabilities_part(result=exp_result_1[0])
    noisy_value_2=lu.readout_data_to_state_probabilities_part(result=exp_result_2[0])
    noisy_value_3=lu.readout_data_to_state_probabilities_part(result=exp_result_3[0])
    
    return noisy_value_1,noisy_value_2,noisy_value_3

In [7]:
backend_ideal = QasmSimulator() # Noiseless
run_config_ideal = {'shots': 100000, 'backend': backend_ideal, 'name': 'ideal'}

In [ ]:
gate_set = ['sx','cx','rz','x','id']
for qubit in range(2,15):
    entries = []
    for index_i in range(20):
        ciucuit_depth = random.randint(1, 3)
        circuit = generate_random_circuit(coupling_map,useful_qubits,66,qubit,2,gate_set)
        job_ideal = execute(circuit, **run_config_ideal)
        ideal_value = dict(job_ideal.result().get_counts()) # dict
        # print(circuit.qasm())
        qasm_circuit = circuit.qasm()
        with open(f"./multi_backend_model_data/tianyan_test_qasm/tianyan_{qubit}_qubit_{index_i}.qasm",'w') as file:
                    file.write(qasm_circuit)
        qcis_circuit = QasmToQcis().convert_to_qcis(qasm_circuit)
        noisy_value_1,noisy_value_2,noisy_value_3 = run_tianyan_circuit(qcis_circuit,platform,lab_id)
        # print(noisy_value_1)
        to_append = {}
        to_append['circuit'] = circuit
        to_append['ideal_exp_value'] = ideal_value
        to_append['noisy_exp_values_1'] = noisy_value_1
        to_append['noisy_exp_values_2'] = noisy_value_2
        to_append['noisy_exp_values_3'] = noisy_value_3
        entries.append(to_append)
    with open(f"./multi_backend_model_data/tianyan_test_data/tianyan_{qubit}_qubit.pk", "wb") as pk_file:
            pickle.dump(entries, pk_file)

In [8]:
gate_set = ['sx','cx','rz','x','id']
qubit = 25
entries = []
for index_i in range(5):
    # ciucuit_depth = random.randint(1, 3)
    circuit = generate_random_circuit(coupling_map,useful_qubits,66,qubit,2,gate_set)
    job_ideal = execute(circuit, **run_config_ideal)
    ideal_value = dict(job_ideal.result().get_counts()) # dict
    # print(circuit.qasm())
    qasm_circuit = circuit.qasm()
    with open(f"./multi_backend_model_data/tianyan_big_test_qasm/tianyan_{qubit}_qubit_{index_i}.qasm",'w') as file:
                file.write(qasm_circuit)
    qcis_circuit = QasmToQcis().convert_to_qcis(qasm_circuit)
    noisy_value_1,noisy_value_2,noisy_value_3 = run_tianyan_circuit(qcis_circuit,platform,lab_id)
    # print(noisy_value_1)
    to_append = {}
    to_append['circuit'] = circuit
    to_append['ideal_exp_value'] = ideal_value
    to_append['noisy_exp_values_1'] = noisy_value_1
    to_append['noisy_exp_values_2'] = noisy_value_2
    to_append['noisy_exp_values_3'] = noisy_value_3
    entries.append(to_append)
with open(f"./multi_backend_model_data/tianyan_big_test_data/tianyan_{qubit}_qubit.pk", "wb") as pk_file:
        pickle.dump(entries, pk_file)

In [54]:
gate_set = ['sx','cx','rz','x','id']
circuit = generate_random_circuit(coupling_map,useful_qubits,66,5,5,gate_set)
print(circuit.qasm())
qcis_circuit = QasmToQcis().convert_to_qcis(circuit.qasm())
print(qcis_circuit)

OPENQASM 2.0;
include "qelib1.inc";
qreg q[66];
creg c[5];
rz(3.005800866918804) q[32];
x q[7];
x q[9];
x q[15];
sx q[7];
cx q[9],q[15];
cx q[9],q[15];
cx q[9],q[15];
sx q[7];
x q[9];
rz(0.3981363621320617) q[13];
cx q[7],q[13];
cx q[7],q[13];
sx q[32];
sx q[9];
sx q[15];
measure q[32] -> c[0];
measure q[7] -> c[1];
measure q[9] -> c[2];
measure q[13] -> c[3];
measure q[15] -> c[4];

RZ Q32 3.005800866918804
X2P Q7
X2P Q7
X2P Q9
X2P Q9
X2P Q15
X2P Q15
X2P Q7
Y2M Q15
CZ Q9 Q15
Y2P Q15
Y2M Q15
CZ Q9 Q15
Y2P Q15
Y2M Q15
CZ Q9 Q15
Y2P Q15
X2P Q7
X2P Q9
X2P Q9
RZ Q13 0.3981363621320617
Y2M Q13
CZ Q7 Q13
Y2P Q13
Y2M Q13
CZ Q7 Q13
Y2P Q13
X2P Q32
X2P Q9
X2P Q15
M Q32
M Q7
M Q9
M Q13
M Q15


x = 2*X2P
cx = Y2M+CZ+Y2P
sx = Y2M
rz =RZ


In [16]:
gate_set = ['sx','cx','rz','x','id']
circuit = generate_random_circuit(coupling_map,useful_qubits,66,25,3,gate_set)
job_ideal = execute(circuit, **run_config_ideal)
ideal_value = dict(job_ideal.result().get_counts()) # dict
print(ideal_value)

{'1001000001011010011111001': 47, '1001000001001010101101001': 58, '1000001001011011111101001': 50, '1000000001001110101111001': 56, '1000001001011011011101101': 55, '1010000001001011011100101': 47, '1011001001001111101110001': 39, '1010001001011011111100101': 38, '1000001001011011011110001': 51, '1000000001011010001111101': 40, '1000001001001011111110101': 60, '1001001001011010011110101': 35, '1000001001001110011101101': 49, '1001000001011010001100001': 51, '1001000001001010111110001': 50, '1010000001011111011100101': 59, '1001000001001110101110101': 56, '1001000001011111101101001': 53, '1001001001011011111100001': 43, '1001000001001011011111101': 48, '1000001001001010011101001': 47, '1001001001011011011110101': 53, '1010000001001011001111001': 44, '1011001001011011111100101': 51, '1010001001011110111101101': 49, '1011001001011110011100001': 58, '1010001001011011101111001': 44, '1010000001001111011101101': 42, '1000001001011010111100101': 55, '1000001001011110111110001': 47, '10100010

In [8]:
from qiskit.transpiler import CouplingMap
CM = CouplingMap(coupling_map)
print(type(CM))

<class 'qiskit.transpiler.coupling.CouplingMap'>


In [12]:
backend_ideal = QasmSimulator() # Noiseless
run_config_ideal = {'shots': 100000, 'backend': backend_ideal, 'name': 'ideal'}

In [52]:
gate_set = ['sx','cx','rz','x','id']
entries = []
qubit  = 25
index_i = 0
while(index_i < 5):
    circuit = generate_random_circuit_25(coupling_map,useful_qubits,66,25,1,gate_set)
    job_ideal = execute(circuit, **run_config_ideal)
    ideal_value = dict(job_ideal.result().get_counts()) # dict
    for key in ideal_value:
        ideal_value[key] /= 100000
    max_3 = sorted(ideal_value.items(), key=lambda x: x[1], reverse=True)[:3]
    sum_of_top_3_values = sum(sorted(ideal_value.values(), reverse=True)[:3])
    print(max_3)
    print(sum_of_top_3_values)
    if(sum_of_top_3_values > 0.8):
        index_i +=1
        print('yes')
        qasm_circuit = circuit.qasm()
        with open(f"./multi_backend_model_data/tianyan_big_test_qasm/tianyan_{qubit}_qubit_{index_i}.qasm",'w') as file:
                    file.write(qasm_circuit)
        qcis_circuit = QasmToQcis().convert_to_qcis(qasm_circuit)
        noisy_value_1,noisy_value_2,noisy_value_3 = run_tianyan_circuit(qcis_circuit,platform,lab_id)
        # print(noisy_value_1)
        to_append = {}
        to_append['circuit'] = circuit
        to_append['ideal_exp_value'] = ideal_value
        to_append['noisy_exp_values_1'] = noisy_value_1
        to_append['noisy_exp_values_2'] = noisy_value_2
        to_append['noisy_exp_values_3'] = noisy_value_3
        entries.append(to_append)
    with open(f"./multi_backend_model_data/tianyan_big_test_data/tianyan_{qubit}_qubit.pk", "wb") as pk_file:
            pickle.dump(entries, pk_file)

[('0000000000000100100010011', 0.06347), ('0000000000000100101110011', 0.06335), ('0000000000000100100110011', 0.06315)]
0.18996999999999997
[('0000000000001000000100000', 0.25119), ('0000000000001000001100000', 0.25016), ('0000000000001000000000000', 0.24983)]
0.75118
[('0000000000000001100000000', 0.25269), ('0000000000000000100001000', 0.25225), ('0000000000000001100001000', 0.24773)]
0.75267
[('0000000000000000111110000', 0.25115), ('0000000000000000111110001', 0.2511), ('0000000000000000111100001', 0.25016)]
0.75241
[('0000000000000000000100011', 0.50204), ('0000000000000000000100001', 0.49796)]
1.0
yes
[('0000000000001010111001001', 0.5011), ('0000000000001000111001001', 0.4989)]
1.0
yes
[('0000000000000000101000101', 0.5017), ('0000000000000000101010101', 0.4983)]
1.0
yes
[('0000000000000100000001001', 0.25191), ('0000000000000100000000001', 0.2502), ('0000000000000100100000001', 0.24941)]
0.7515200000000001
[('0000000000000010000100010', 1.0)]
1.0
yes
[('00000000000000100100010

In [ ]:
qasm = circuit.qasm()

In [49]:
# 示例字典
my_dict = {'a': 10, 'b': 20, 'c': 30, 'd': 40, 'e': 50}

# 求值的前3个的和
sum_of_top_3_values = sum(sorted(my_dict.values(), reverse=True)[:3])

print(sum_of_top_3_values)  # 输出值的前3个的和


120
